In [10]:
import numpy as np
from PIL import Image
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import torch.nn as nn
import torchvision
import torch.optim as optim

Metoda za dobavljanje slika

In [11]:
def get_images(root, last_image_index):
    train_images = []
    test_images = []
    
    for i in range(1, last_image_index):
      name = str(i) + ".JPG"
      img = Image.open(root + '/' + name)
      basewidth = 128
      wpercent = (basewidth/float(img.size[0]))
      hsize = int((float(img.size[1])*float(wpercent)))
      img = img.resize((basewidth,hsize), Image.ANTIALIAS)
      a = np.asarray(img, dtype='float32')
      a = a * 256
      a = a - 2048
      a[a<0] = 0
      a /= 65536
      a = torch.from_numpy(a)
      a = a.transpose(0, 2)
      if i < int(0.8*last_image_index):
        train_images.append(a)
      else:
        test_images.append(a)
        
    return train_images, test_images

In [12]:
from google.colab import drive
drive.mount('/content/drive')
train_images, test_images = get_images("/content/drive/My Drive/JPG", 1708)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Priprema slika za neuronsku mrežu

In [15]:
def get_labels(root):
    labels = []
    
    labels_file = open(root, 'r')
    lines = labels_file.readlines()
    
    for line in lines:
        rgb = []
        line = line.split(" ")
        rgb.append(float(line[0]))
        rgb.append(float(line[1]))
        rgb.append(float(line[2]))
        
        labels.append(rgb)
        
    return labels

train_images = torch.stack(train_images)
test_images = torch.stack(test_images)

labels = torch.FloatTensor(get_labels("/content/drive/My Drive/dataset/cube+_gt.txt"))

train_labels = labels[:int(0.8*len(labels))]
test_labels = labels[int(0.8*len(labels)):]
print(train_labels.shape, test_labels.shape)

torch.Size([1365, 3]) torch.Size([342, 3])


In [16]:
train_images = train_images.type(torch.FloatTensor)
test_images = test_images.type(torch.FloatTensor)

train_labels = train_labels.type(torch.FloatTensor)
test_labels = test_labels.type(torch.FloatTensor)

print(train_images.shape, train_labels.shape)

torch.Size([1365, 3, 128, 85]) torch.Size([1365, 3])


In [17]:
train_dataset = TensorDataset(train_images, train_labels)
test_dataset = TensorDataset(test_images, test_labels)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

Definiranje klase neuronske mreže

In [18]:
 class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    self.conv1 = nn.Conv2d(3, 16, kernel_size=5)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=5)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=5)

    self.fc1 = nn.Linear(5376, 1000)
    self.fc2 = nn.Linear(1000, 200)
    self.fc3 = nn.Linear(200, 3)

    self.batch = nn.BatchNorm1d(1000)
    
    self.maxpool = nn.MaxPool2d(kernel_size=2)
    self.relu = nn.ReLU(inplace=True)

  def forward(self, x):
    x = self.conv1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.conv2(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.conv3(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = x.view(-1, 5376)
    x = self.fc1(x)
    x = self.relu(x)

    x = self.batch(x)

    x = self.fc2(x)
    x = self.relu(x)

    x = self.fc3(x)
    x = self.relu(x)
    
    return torch.tanh(x)

In [19]:
n_epochs = 10
learning_rate = 0.00005
device = 'cpu'

Metoda za korak treniranja

In [20]:
def train_step(network, train_data, epoch, device):
  losses = []
  counter = []

  network.train()
  for idx, (data, target) in enumerate(train_data):
    data = data.to(device)
    target = target.to(device)

    network.zero_grad()
    output = network(data)
    loss = F.mse_loss(output, target)
    loss.backward()
    optimizer.step()

    if idx % 5 == 0:
      print('Train Epoch: {:2d}, ({:2.0f}%), Loss: {:.5f}'.format(
          epoch, 100*idx*64/len(train_data.dataset), loss
      ))
      losses.append(loss)
      counter.append(idx*64 + (epoch-1)*len(train_data.dataset))

  return losses, counter

Metoda za testiranje

In [21]:
def test(network, test_data, device):
  network.eval()

  loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_data:
      data = data.to(device)
      target = target.to(device)

      output = network(data)
      loss += F.mse_loss(output, target, reduction="sum").item()

  loss /= len(test_data.dataset)
  accuracy = 100 * correct / len(test_data.dataset)

  return loss, accuracy

In [22]:
def train(network, train_data, test_data, device='cpu'):
  test_loss = []
  test_acc = []
  train_loss = []
  counter = []

  # testiramo pocetni model
  loss_t, acc_t = test(network, test_data, device)
  test_loss.append(loss_t)
  test_acc.append(acc_t)

  for epoch in range(1, n_epochs+1):
    # korak i testsad 
    loss, cnt = train_step(network, train_data, epoch, device)
    loss_t, acc_t = test(network, test_data, device)

    test_loss.append(loss_t)
    test_acc.append(acc_t)
    train_loss.append(loss)
    counter.append(cnt)

  return test_loss, test_acc, train_loss, counter



Kreiranje i treniranje mreže

In [23]:
network = Net()

optimizer = optim.Adam(network.parameters(), lr=learning_rate)
test_loss, test_acc, train_loss, counter = train(network, train_loader, test_loader, device)

Train Epoch:  1, ( 0%), Loss: 0.09981
Train Epoch:  1, (23%), Loss: 0.05669
Train Epoch:  1, (47%), Loss: 0.04511
Train Epoch:  1, (70%), Loss: 0.03817
Train Epoch:  1, (94%), Loss: 0.02610
Train Epoch:  2, ( 0%), Loss: 0.02289
Train Epoch:  2, (23%), Loss: 0.01905
Train Epoch:  2, (47%), Loss: 0.01938
Train Epoch:  2, (70%), Loss: 0.01590
Train Epoch:  2, (94%), Loss: 0.01394
Train Epoch:  3, ( 0%), Loss: 0.01060
Train Epoch:  3, (23%), Loss: 0.00795
Train Epoch:  3, (47%), Loss: 0.00633
Train Epoch:  3, (70%), Loss: 0.00617
Train Epoch:  3, (94%), Loss: 0.00761
Train Epoch:  4, ( 0%), Loss: 0.00538
Train Epoch:  4, (23%), Loss: 0.00325
Train Epoch:  4, (47%), Loss: 0.00377
Train Epoch:  4, (70%), Loss: 0.00485
Train Epoch:  4, (94%), Loss: 0.00387
Train Epoch:  5, ( 0%), Loss: 0.00391
Train Epoch:  5, (23%), Loss: 0.00392
Train Epoch:  5, (47%), Loss: 0.00184
Train Epoch:  5, (70%), Loss: 0.00278
Train Epoch:  5, (94%), Loss: 0.00243
Train Epoch:  6, ( 0%), Loss: 0.00216
Train Epoch: